In [7]:
import numpy as np
import pandas as pd
from skcuda import cublas
from pycuda import gpuarray
import pycuda.autoinit
import time
from pycuda.compiler import SourceModule
import pycuda.driver as cuda
import ThrustRTC as trtc
import numpy as np
import time
from matplotlib import pyplot
import pandas as pd
from sklearn import metrics
from collections import Counter
import random

FLOAT_MAX = 1e10
# np.random.seed(1)
# random.seed(1)

In [8]:
# 求平方
get_square = SourceModule(
'''
__global__ void x_square(float *x, int &n)
{
    const int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx<n)
        x[idx] = x[idx] * x[idx];
}

__global__ void y_square(float *y, int &m)
{
    const int idx = threadIdx.x;
    if(idx<m)
        y[idx] = y[idx] * y[idx];

}
'''
)


# 求和
square_sum = SourceModule(
'''

__global__ void sum_row_x(float *s_x, float *self_sum1, int &numSample, int &k, int &dim)
{
    const int idx1 = threadIdx.x + blockIdx.x * blockDim.x;
    //const int idx2 = threadIdx.y + blockIdx.y * blockDim.y;

    __shared__ float sdata[1024];
    const int tid = threadIdx.x;
    sdata[tid] = s_x[idx1];
    __syncthreads();
    // do reduction in shared mem
    for(unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
    sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
    }
    // write result for this block to global mem
    if(tid == 0)  self_sum1[blockIdx.x] = sdata[0];
    
    
}



__global__ void sum_row_y(float *s_y, float *self_sum2, int &numSample, int &k, int &dim)
{
    //const int idx1 = threadIdx.x + blockIdx.x * blockDim.x;
    const int idx2 = threadIdx.y + blockIdx.y * blockDim.y;

    __shared__ float sdata[1024];
    const int tid = threadIdx.y;
    sdata[tid] = s_y[idx2];
    __syncthreads();
    // do reduction in shared mem
    for(unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
    sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
    }
    // write result for this block to global mem
    if(tid == 0)  self_sum2[blockIdx.x] = sdata[0];
    
}

__global__ void sum_row_xy(float *s_x, float *s_y, float *self_sum1, float *self_sum2, int &numSample, int &k, int &dim)
{
    const int idx = threadIdx.x + blockIdx.x * blockDim.x;
    //const int idy = threadIdx.y + blockIdx.y * blockDim.y;
    
    
    if(idx < k)
        for(int i=0;i<dim;i++)
            self_sum2[idx] += s_y[idx*dim+i];


    if(idx < numSample + k && idx >= k)    
        for(int j=0;j<dim;j++)    
            self_sum1[idx - k] += s_x[(idx - k)*dim+j];    
    
    __syncthreads(); 
}

__global__ void sum_total(float *self_sum1, float *self_sum2, float *result, int &k, int &numSample)
{
    const int idx1 = threadIdx.x + blockIdx.x * blockDim.x;
    int quo = idx1 / k;
    int mod = idx1 % k;
    
    if(idx1 < numSample * k)
    
        result[idx1] = self_sum1[quo] + self_sum2[mod];

    __syncthreads();     

}

__global__ void sum_total_new(float *s_x, float *s_y, float *result, int &numSample, int &k, int &dim)
{
    const int idx = threadIdx.x + blockDim.x * blockIdx.x;
    float self_sum1[50000] = {0};
    float self_sum2[5] = {0};
    if(idx < k)
    {
        for(int i=0;i<dim;i++)
            self_sum2[idx] += s_y[idx*dim+i];
    }

    if(idx < numSample + k && idx >= k)    
        for(int j=0;j<dim;j++)    
            self_sum1[idx - k] += s_x[(idx - k)*dim+j];    
    
    //__syncthreads(); 
    if(idx >= numSample + k && idx < numSample * 2 + k)
        for(int ii=0; ii<k; ii++)
        
            result[ii + (idx - numSample - k) * k] = self_sum1[idx - numSample - k] + self_sum2[ii];
        //flattened
        
    __syncthreads(); 
}

'''
)



# 判断距离数据是否为有效数据
check = SourceModule(
   '''
__global__ void check_if_valid(float *g_data, int &length){
    float temp_data[512]={0};
    
    unsigned int tid = threadIdx.x;
    unsigned int i= blockIdx.x*blockDim.x+ threadIdx.x;
    if(tid < length)
    {
    temp_data[tid] = g_data[i];
    }
    __syncthreads();    
    if (tid < length)     
    g_data[i] = (temp_data[tid] > 0)? temp_data[tid]:0;
    
    __syncthreads();
    }
  
   '''
)


# shared_memory.max()=48KB(12000 int/float32)
find = SourceModule(
'''
__global__ void find_range(float *cluster, float *result)
{
    __shared__ float sdata[512];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x*blockDim.x+ threadIdx.x;
    sdata[tid] = cluster[i];
    __syncthreads();
    if(sdata[tid] != sdata[tid+1] and tid < 499) // 统计各簇中元素的个数
        result[int(sdata[tid])] = i+1;
    __syncthreads();
}


// 下面两个函数可以合并成一个函数
// min函数在cuda中复现，使用reduction思想
__global__ void find_minimum(float *list, float *result_value, int &length)
{
    __shared__ float sdata[512];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;    
    if(tid < length)
    {
    sdata[tid] = list[i];    
    }

    __syncthreads();
    
    if(tid < length)
        for(unsigned int s=blockDim.x/2; s>0; s>>=1) {
        if (tid < s)
        {
        sdata[tid] = (sdata[tid] < sdata[tid + s])? sdata[tid] : sdata[tid + s];        
        }
        __syncthreads();
        }
    if(tid == 0)
    result_value[blockIdx.x] = sdata[0]; 

}


__global__ void find_minimum_index(float *list, int *result_idx, int &length)
{
    __shared__ float sdata[512];
    __shared__ int ssdata[512];
    unsigned int tid = threadIdx.x;
    unsigned int i= blockIdx.x*blockDim.x+ threadIdx.x;    
    if(tid < length)
    {
    sdata[tid] = list[i];
    ssdata[tid] = tid;
    }

    __syncthreads();
    if(tid < length)
        for(unsigned int s=blockDim.x/2; s>0; s>>=1) {
        if (tid < s) 
            ssdata[tid] = (sdata[tid] < sdata[tid + s])? ssdata[tid]:ssdata[tid+s];
            __syncthreads();
        if(tid < s)
            sdata[tid] = (sdata[tid] < sdata[tid + s])? sdata[tid]:sdata[tid+s];    
            __syncthreads();
        }

    if(tid == 0)
    result_idx[blockIdx.x] = ssdata[0];
}
'''
)



In [47]:
# matrix1 is dataset, matrix2 is centroids
#数据点得分批去输
def get_xy_square(matrix1, matrix2, cluster_k):
    n = matrix1.shape[0]*matrix1.shape[1]
    grid_dim = n // 1024 + 1
    n = np.int32(n)
    n_gpu = cuda.mem_alloc(n.nbytes)
    cuda.memcpy_htod(n_gpu, n)

    x = matrix1.flatten().astype(np.float32)
    #print(x.shape)
    x_gpu = cuda.mem_alloc(x.nbytes)
    cuda.memcpy_htod(x_gpu, x)
    x_func = get_square.get_function("x_square")
    
    # we have a flexible block assignment
    x_func(x_gpu, n_gpu, block=(1024,1,1), grid=(grid_dim,1))
    # you can print the variables if you want
    # cuda.memcpy_dtoh(x, x_gpu)
    # x = np.reshape(x, (matrix1.shape[0], matrix1.shape[1]))
    
    m = matrix2.shape[0] * matrix2.shape[1]
    # determining grid size

    grid_dim = m // 1024 + 1

    m = np.int32(m)
    m_gpu = cuda.mem_alloc(m.nbytes)
    cuda.memcpy_htod(m_gpu, m)

    y = matrix2.flatten().astype(np.float32)
    y_gpu = cuda.mem_alloc(y.nbytes)
    cuda.memcpy_htod(y_gpu, y)
    y_func = get_square.get_function("y_square")
    

    y_func(y_gpu, m_gpu, block=(1024,1,1), grid=(grid_dim,1))
    # you can print the variables if you want 
    # cuda.memcpy_dtoh(y, y_gpu)
    # y = np.reshape(y, (matrix2.shape[0], matrix2.shape[1]))
    
    self_sum1 = np.zeros((matrix1.shape[0], ), dtype = np.float32)
    block_dim = len(self_sum1) // 1024 + 1
    self_sum2 = np.zeros((matrix2.shape[0], ), dtype = np.float32)
    self_sum1_gpu = cuda.mem_alloc(self_sum1.nbytes)
    self_sum2_gpu = cuda.mem_alloc(self_sum2.nbytes)
    cuda.memcpy_htod(self_sum1_gpu, self_sum1)
    cuda.memcpy_htod(self_sum2_gpu, self_sum2)
    
    
    numSample = np.int32(matrix1.shape[0])
    numSample_gpu = cuda.mem_alloc(numSample.nbytes)
    cuda.memcpy_htod(numSample_gpu, numSample)
    
    k = np.int32(cluster_k)
    k_gpu = cuda.mem_alloc(k.nbytes)
    cuda.memcpy_htod(k_gpu, k)
    
    dim = np.int32(matrix1.shape[1])
    dim_gpu = cuda.mem_alloc(dim.nbytes)
    cuda.memcpy_htod(dim_gpu, dim)

    result = np.zeros((matrix2.shape[0]*matrix1.shape[0], ), dtype = np.float32)
    block_dim_result = len(result) // 1024 + 1
    result_gpu = cuda.mem_alloc(result.nbytes)
    cuda.memcpy_htod(result_gpu, result)

    #sum_func_row_x = square_sum.get_function("sum_row_x")

    #sum_func_row_x(x_gpu, self_sum1_gpu, numSample_gpu, k_gpu, dim_gpu, block = (1024, 1, 1), grid = (block_dim,1))

    #sum_func_row_y = square_sum.get_function("sum_row_y")
   # sum_func_row_y(y_gpu, self_sum2_gpu, numSample_gpu, k_gpu, dim_gpu, block = (1, int(cluster_k + 1), 1), grid = (1,40))
    
    # optimization v1.0
    sum_func_row = square_sum.get_function('sum_row_xy')

    grid_dim_new = 50000 // 1024 + 1 # 49
    sum_func_row(x_gpu, y_gpu, self_sum1_gpu, self_sum2_gpu, numSample_gpu, k_gpu, dim_gpu, block = (1024,1,1), grid = (grid_dim_new,1))
    
    sum_func_total = square_sum.get_function("sum_total")
    sum_func_total(self_sum1_gpu, self_sum2_gpu, result_gpu, k_gpu, numSample_gpu, block=(1024,1,1), grid=(block_dim_result,1))
    
    # optimization v2.0??? 
    #sum_func_total_new = square_sum.get_function("sum_total_new")
    #sum_func_total_new(x_gpu, y_gpu, result_gpu, numSample_gpu, k_gpu, dim_gpu, block=(512,1,1),grid=(200,1))
    
    result_cpu = np.empty_like(result)
    cuda.memcpy_dtoh(result_cpu, result_gpu)
    result_cpu = np.reshape(result_cpu, (matrix1.shape[0], matrix2.shape[0]))
    result_cpu = np.transpose(result_cpu)
    
    return result_cpu


# 计算与质心之间的距离
def dis_computation(matrix1, matrix2, matrix3):
    """matrix 1 is centroids, 
    matrix 2 is dataset.transpose, 
    matrix 3 is result"""
    
    matrix1 = matrix1.astype(np.float32)
    matrix2 = matrix2.astype(np.float32)
    matrix3 = matrix3.astype(np.float32)
    
    ldc = matrix3.shape[1]
    m = matrix2.shape[1]
    n = matrix1.shape[0]

    C = matrix3.flatten()
    #the above is matrix C
    ldb = matrix1.shape[1]
    k = matrix1.shape[1]
    B = matrix1.flatten()
    #the above is matrix B

    lda = matrix2.shape[1]
    A = matrix2.flatten()
    # the above is matrix A
    transa = 'n'
    transb = 'n'
    alpha = -2
    beta = 1
    #copy data into GPU
    A_gpu = gpuarray.to_gpu(A)
    B_gpu = gpuarray.to_gpu(B)
    C_gpu = gpuarray.to_gpu(C)
    # computing matrix
    h = cublas.cublasCreate()
    cublas.cublasSgemm(h, transa, transb, m, n, k, alpha, A_gpu.gpudata, lda, B_gpu.gpudata, ldb, beta, C_gpu.gpudata, ldc)
    cublas.cublasDestroy(h)
    
    C = C_gpu.get() # flattened
    C = np.reshape(C, (n, m))
    
    C = C.transpose() # 494021 * 5
    C = C.flatten()
    return C
 
    

def cluster_assignment(C):
# 实质上这里也是在排序
    minDist = np.amin(C, axis = 1)
    minDistIdx = np.argmin(C, axis = 1)
    
    cluster_assignment = np.zeros((len(minDistIdx),2))
    #for i in range(len(minDistIdx)):
    cluster_assignment[:,0] = minDistIdx
    cluster_assignment[:,1] = minDist
    
    return cluster_assignment



def sort_by_key_without_count(clusterAssment):
    cluster = clusterAssment[:,0].astype(np.float32)
    cluster_gpu = trtc.device_vector_from_numpy(cluster)
    n = cluster_gpu.size()
    index = trtc.device_vector("int32_t", n)
    trtc.Sequence(index)
    trtc.Sort_By_Key(cluster_gpu, index)
    index_cpu = index.to_host()
    cluster_cpu = cluster_gpu.to_host()
    return index_cpu, cluster_cpu




def sort_gpu(index_cpu, cluster_cpu, dataset, k, placement):
    if(dataset.shape[0] > 250000):
        cluster1 = cluster_cpu[0:250000].astype(np.float32)
        cluster2 = cluster_cpu[250000:-1].astype(np.float32)
        result1 = np.zeros((k-1,),dtype=np.float32)
        result2 = np.zeros((k-1,),dtype=np.float32)
        find_func = find.get_function("find_range")
        find_func(cuda.In(cluster1),cuda.Out(result1), block=(500,1,1),grid=(500,1))
        find_func(cuda.In(cluster2),cuda.Out(result2), block=(500,1,1),grid=(500,1))
        # 取置信值，这里的代码可以优化一下
        # 如果数据量控制在25w以内效果最佳
        for j in range(len(result2)):
            if(result2[j] != 0):
                result2[j] += 250000 # 分批处理，第一批计数从0开始，第二批从250000开始
        for i in range(len(result2)-1):
            if(result2[i] > result2[i+1]): # 消除异常值
                result2[i] = 0
        starting_points = [0]
        for jj in range(len(result2)):
            starting_points.append(max(result1[jj], result2[jj]))
        starting_points.append(dataset.shape[0]+1)
        data_in_cluster = []
        start = int(starting_points[placement])
        end = int(starting_points[placement+1])
        data_in_cluster = dataset[index_cpu[start:end],:]
        data_in_cluster = np.array(data_in_cluster)
    else:
        cluster1 = cluster_cpu[0:250000].astype(np.float32)
        result1 = np.zeros((k-1,),dtype=np.float32)
        find_func = find.get_function("find_range")
        find_func(cuda.In(cluster1),cuda.Out(result1), block=(500,1,1),grid=(500,1))
        starting_points = [0]
        for jj in range(len(result1)):
            starting_points.append(result1[jj])
        starting_points.append(dataset.shape[0]+1)
        data_in_cluster = []
        start = int(starting_points[placement])
        end = int(starting_points[placement+1])
        data_in_cluster = dataset[index_cpu[start:end],:]
        data_in_cluster = np.array(data_in_cluster)
    return data_in_cluster



def cluster_assignment_gpu(C):
    C = C.astype(np.float32)
    length = np.int32(C.shape[1])
    length_gpu = cuda.mem_alloc(length.nbytes)
    cuda.memcpy_htod(length_gpu, length)
    result = np.zeros((C.shape[0],), dtype = np.float32)
    # 找到距离最近的质心点
    find_func = find.get_function("find_minimum")
    find_func(cuda.In(C),cuda.Out(result),length_gpu,block=(C.shape[1],1,1),grid=(C.shape[0],1))
    
    result_idx = np.argmin(C, axis = 1)    
    cluster_assignment = np.zeros((len(result_idx),2))
    cluster_assignment[:,0] = result_idx
    cluster_assignment[:,1] = result
    return cluster_assignment
    
    
#这个函数的进一步优化我还没有想好
def centroids_update(pointsInCluster, k):
    if(pointsInCluster.size == 0):
        return
    dim = pointsInCluster.shape[1]
    length = pointsInCluster.shape[0]
    sum_cpu = []
    for i in range(dim):
        p = pointsInCluster.transpose()[i].astype(np.float32)
        pointsInCluster_test_gpu = trtc.device_vector_from_numpy(p)
        sum_cpu.append(trtc.Reductions.Reduce(pointsInCluster_test_gpu))
    sum_cpu = np.array(sum_cpu)

    kernel = trtc.Kernel(['sum', 'mean', 'dim','length'],
    '''
        const int idx = threadIdx.x;
        if(idx < dim)
            mean[idx] = sum[idx]/length;
    ''')

    
    sum_gpu = trtc.device_vector_from_numpy(sum_cpu)
    mean_cpu = np.zeros((dim,), dtype = np.float32)
    mean_gpu = trtc.device_vector_from_numpy(mean_cpu)
    kernel.launch(1,dim,[sum_gpu, mean_gpu, trtc.DVInt32(dim), trtc.DVInt32(length)])
    mean_cpu = mean_gpu.to_host()
    return mean_cpu


def get_centroids_gpu(points, k):
    m, n = np.shape(points)
    cluster_centers = np.zeros((k , n))
    # 1、随机选择一个样本点为第一个聚类中心
    index = np.random.randint(0, m)
    cluster_centers[0, ] = np.copy(points[index, ])
    # 2、初始化一个距离的序列
    #d = [0.0 for _ in range(m)]

    for i in range(1, k):
        sum_all = 0
        #for j in range(m):
            # 3、对每一个样本找到最近的聚类中心点
        matrix1 = cluster_centers[0:i,]
        distance_matrix = []
        for pack in range(m // 50000 + 1): # well this is tricky
            subset = points[pack*50000:min(m, (pack+1)*50000)]
            matrix2 = subset.transpose()
            matrix3 = get_xy_square(subset, cluster_centers[0:i,], i)
            distance_matrix.append(dis_computation(matrix1, matrix2, matrix3))
        distance_matrix = np.array(distance_matrix)
        distance_matrix_merge = np.vstack((distance_matrix[0], distance_matrix[1]))
        if(len(distance_matrix) > 2):
            for ii in range(2,len(distance_matrix)):
                distance_matrix_merge = np.vstack((distance_matrix_merge, distance_matrix[ii]))

        distance_matrix_merge = distance_matrix_merge.reshape((m, i))           
        d = np.amin(distance_matrix_merge, axis = 1)
            # 4、将所有的最短距离相加
        sum_all = np.sum(d)
        # 5、取得sum_all之间的随机值
        sum_all *= random.random()
        # 6、获得距离最远的样本点作为聚类中心点
        for j, di in enumerate(d):
            sum_all -= di
            if sum_all > 0:
                continue
            cluster_centers[i] = np.copy(points[j, ])
            break
    return cluster_centers

In [48]:
find_minimum = SourceModule(
'''
__global__ void find_min(float *list, float *result_value, int *result_idx, int &length, int &n)
{
    __shared__ float sdata[1024];
    __shared__ int ssdata[1024];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;   

    
    if(tid < length)
    {
    sdata[tid] = list[i]; // store data
    ssdata[tid] = tid; // store index
    }
    unsigned int s;    

    __syncthreads();
    if(tid < length)
    {
        for(s = blockDim.x/2; s>0; s>>=1) {
        if (tid < s) 
        {
            ssdata[tid] = (sdata[tid] < sdata[tid + s])? ssdata[tid]:ssdata[tid+s];
            __syncthreads();
            sdata[tid] = (sdata[tid] < sdata[tid + s])? sdata[tid]:sdata[tid+s];
            __syncthreads();
        }
            
        __syncthreads();

}
        if(s == 0)
        {
            ssdata[0] = (sdata[0] < sdata[n-1])? ssdata[0]:ssdata[n-1];
            sdata[0] = (sdata[0] < sdata[n-1])? sdata[0]:sdata[n-1];
            __syncthreads();
        }    
    }
       

    if(tid == 0)
    {
    result_idx[blockIdx.x] = ssdata[0];
    result_value[blockIdx.x] = sdata[0];     
    }

}


'''
)


def cluster_assignment_new(C, k):

    C = C.astype(np.float32) # flattened
    length = np.int32(k)
    length_gpu = cuda.mem_alloc(length.nbytes)
    cuda.memcpy_htod(length_gpu, length)
    result_value = np.zeros((len(C)//k,),dtype = np.float32)
    result_idx = np.zeros((len(C)//k,),dtype = np.int32)
    n = k
    divider = k
    while(not(divider % 2)):
        n = divider / 2
        divider /= 2
    n = np.int32(n)
    n_gpu = cuda.mem_alloc(n.nbytes)
    cuda.memcpy_htod(n_gpu, n)
    find_func_hybrid = find_minimum.get_function("find_min")    
    find_func_hybrid(cuda.In(C),cuda.Out(result_value),cuda.Out(result_idx),length_gpu, n_gpu, block=(k,1,1),grid=(len(C)//k,1))
    cluster_assignment = np.zeros((len(result_idx),2))
    #for i in range(len(minDistIdx)):
    cluster_assignment[:,0] = result_idx.astype(np.int32)
    cluster_assignment[:,1] = result_value
    return cluster_assignment

dis_mat = np.loadtxt('distance_mat.csv',delimiter = ',')
dis_mat = dis_mat.flatten()
cluster_assignment_new(dis_mat, 5)

array([[0.        , 0.        ],
       [0.        , 0.00667381]])

In [51]:
# cpu的版本
def euclDistance(vector1,vector2):
    return np.sqrt(sum(np.power(vector2-vector1,2)))#power计算次方

##初始化数据的中心点，k表示聚类中心数
##随机生成k个聚类中心
def initCentroids(dataset,k):
    numSample,dim=dataset.shape
    centroids=np.zeros((k,dim))
    for i in range(k):
        #index=int(np.random.uniform(0,numSample))#随机生成数
        index = int(i*10000)
        centroids[i,:]=dataset[index,:]
    return centroids

##kmean算法
def kmeans(dataset,k):
    numSample=dataset.shape[0]
    #生成新的两列数组，保存聚类信息
    # 第一列表示所属聚类中心，第二列表示与中心的误差
    clusterAssment=np.zeros((numSample,2))#这里dtype就默认
    clusterChanged=True

## step1 初始化聚类中心
    centroids=initCentroids(dataset,k)
    #centroids=get_centroids_gpu(dataset,k)
    storage = np.ones((numSample,))
    storage = storage * k
    itr = 0
    t4centupdate = 0
    index_all = [i for i in range(dataset.shape[0])]
    omit = [] # 在聚类进程中需要忽略的元素
    while (clusterChanged):
        itr += 1

        #clusterChanged=False
        
        count = 0
        for cnt in range(len(storage)):           
            if(storage[cnt] != clusterAssment[cnt,0]):
                count += 1
                if(count > 10):
                    clusterChanged = True
                    break
            else:
                clusterChanged = False

        storage = clusterAssment[:,0]
        #二重循环：对所有数据点，与k个聚类中心计算距离
        #并保存标签与距离
        matrix1 = centroids
        subset = dataset[0:min(numSample, 50000)]
        matrix2 = subset.transpose()
        matrix3 = get_xy_square(subset, centroids, k)
        distance_mat = dis_computation(matrix1, matrix2, matrix3)
        
        clusterAssment1 = clusterAssment # buffer
        if(numSample > 50000): 
            for pack in range(1, dataset.shape[0] // 50000 + 1): # well this is tricky
                subset = dataset[pack*50000:min(numSample, (pack+1)*50000)]
                matrix2 = subset.transpose()
                matrix3 = get_xy_square(subset, centroids, k)
                
                distance_mat = np.hstack((distance_mat, dis_computation(matrix1, matrix2, matrix3)))
            #print(distance_mat[0:11])
            #distance_matrix = np.vstack((distance_matrix[0],distance_matrix[1],distance_matrix[2],distance_matrix[3],distance_matrix[4],distance_matrix[5],distance_matrix[6],distance_matrix[7]))
            #distance_matrix_merge = np.vstack((distance_mat[0], distance_mat[1]))
            #if(len(distance_mat) > 2):
            #    for i in range(2,len(distance_mat)):
            #        distance_matrix_merge = np.vstack((distance_matrix_merge, distance_mat[i]))

            #distance_matrix1 = distance_matrix_merge.reshape((dataset.shape[0], k))
            #distance_matrix1.flatten()


        else:
            matrix1 = centroids
            matrix2 = dataset.transpose()
            matrix3 = get_xy_square(dataset, centroids, k)
            distance_matrix = dis_computation(matrix1, matrix2, matrix3)
            #clusterAssment = cluster_assignment(distance_matrix)
        clusterAssment = cluster_assignment_new(distance_mat, k)
        #print(clusterAssment)
        #temp_clusterAssment = cluster_assignment_new(distance_matrix)
        #np.savetxt('temp_clusterAssment.csv', temp_clusterAssment)
        #print((clusterAssment[:,0] == temp_clusterAssment[:,0]).all())
        #np.savetxt('clusterAssment.csv',clusterAssment)

## step4 循环结束后更新聚类中心
        #start = time.time()
        #index, cluster, count_dict = sort_by_key(clusterAssment)
        #index, cluster = sort_by_key_without_count(clusterAssment)
        #collection = []
        for i in range(k):
            comp = np.nonzero(clusterAssment[:,0] == i)[0] # 当前状态的聚类情况
            comp1 = np.nonzero(clusterAssment1[:,0] == i)[0] # 上一状态的聚类情况
            if(len(comp) == len(comp1) and (comp == comp1).all()):
                #print('In Progress')
                #s = time.time()
             # 当前簇内元素没有改变？如果没有改变，下次聚类忽略该簇
                #if(i != k):
                #    k_content = np.nonzero(clusterAssment[:,0] == k)[0]
                #    clusterAssment[comp, 0] = np.ones(len(clusterAssment[comp,0]),) * (k-1)
                #    clusterAssment[k_content, 0] = np.ones(len(clusterAssment[k_content,0]),) * i

                #k = k-1
                
                #e = time.time()
                #print(e-s)
                #print(k)
                #break
                continue
                
            #pointsInCluster = sort_gpu(index, cluster, dataset, k, i) 
            points_In_k_Cluster_Label=np.nonzero(clusterAssment[:,0]==i)[0]
            pointsInCluster=dataset[points_In_k_Cluster_Label]
            centroids[i, :] = np.mean(pointsInCluster, axis=0)
            #centroids[i,:] = centroids_update(pointsInCluster, k)
        #end = time.time()
        #t4centupdate += (end-start)
    #print(t4centupdate)
        if(itr == 30 or k == 0):            
            break
    #print(itr)
    
    ##循环结束，返回聚类中心和标签信息
    print("Congratulations, cluster complete!")
    return clusterAssment


In [52]:
if __name__=="__main__":
    # print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    # start=time.time()
    ## load data
    dataset=pd.read_csv('kdd_pre_final.csv',sep=',')
    # 真实的标签
    category_real = dataset.loc[:,["clustering"]]
 
    dataset=dataset.loc[:,['duration','protocal_type','service','flag','src_bytes','dst_types','land','wrong_fragment','urgent',
                           'hot','num_falied_logins','logged_in','num_compromised','root_shell','su_attempted','num_root',
                           'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_hot_login','is_guest_login',
                           'count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_error_rate',
                           'diff_error_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate',
                           'dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_diff_src_port_rate',
                           'dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate']]

    
    dataset=np.array(dataset)
    
    ##  k表示聚类中心数
    k = 5
    for _ in range(1):
        start=time.time()
        clusterAssment=kmeans(dataset,k)

        end = time.time()
        print('algorithm (for training) total time: %2f 秒'%(end-start))
    
    category_real = np.array(category_real)
    category = []
    for i in range(dataset.shape[0]):
        category.append(category_real[i][0])
    category = np.array(category)
    category_pre = np.array(clusterAssment[:,0], dtype = np.int32)
    real = Counter(category)
    pre = Counter(category_pre)
    print(real)
    print(pre)
    real = real.most_common()
    pre = pre.most_common()
    for j in range(dataset.shape[0]):
        for nn in range(k):
            if(category[j] == real[nn][0]):
                category[j] = int(pre[nn][0])
    ARI = metrics.adjusted_rand_score(category, category_pre)
    AMI = metrics.adjusted_mutual_info_score(category, category_pre)
    print("调整兰德指数为" + str(ARI))
    print("归一化互信息指数为" + str(AMI))

Congratulations, cluster complete!
algorithm (for training) total time: 5.280585 秒
Counter({2: 391458, 0: 97278, 1: 4107, 4: 1126, 3: 52})
Counter({1: 280824, 0: 87084, 3: 84114, 4: 28191, 2: 13808})
调整兰德指数为0.4447449491311041
归一化互信息指数为0.5101248956364369


In [ ]:
<class 'numpy.float64'>
[[0.         5.38674307 2.04028177 1.24462509 2.00796652]
 [0.00667381 5.34895086 1.93472862 1.22462177 1.85017681]
 [0.01878834 5.24424219 1.84184837 1.17829323 1.70546722]
 ...
 [1.0103941  4.55343056 0.81954193 0.29535484 1.09344769]
 [0.97317219 4.48998356 0.75338745 0.27575207 0.97581577]
 [0.95939016 4.34241199 0.87591743 0.1807909  1.14280033]]
Congratulations, cluster complete!
algorithm (for training) total time: 0.549735 秒
Counter({2: 391458, 0: 97278, 1: 4107, 4: 1126, 3: 52})
Counter({1: 289299, 0: 99335, 4: 70368, 3: 26636, 2: 8383})
调整兰德指数为0.3528364522009577
归一化互信息指数为0.31198447749325825

In [ ]:
2.6293652057647705
15
Congratulations, cluster complete!
algorithm (for training) total time: 7.294799 秒
2.304995059967041
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.783495 秒
2.3896961212158203
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.852052 秒
2.33622407913208
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.742210 秒
2.3348276615142822
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.674991 秒
2.3230175971984863
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.689172 秒
2.330873966217041
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.665528 秒
2.329209327697754
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.608633 秒
2.3477554321289062
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.694374 秒
2.2999649047851562
15
Congratulations, cluster complete!
algorithm (for training) total time: 6.601455 秒
Counter({2: 391458, 0: 97278, 1: 4107, 4: 1126, 3: 52})
Counter({1: 280824, 0: 87084, 3: 84114, 4: 28191, 2: 13808})
调整兰德指数为0.4447449491311041
归一化互信息指数为0.5101248956364369

In [ ]:
# yeast代码

In [ ]:
import numpy as np
import time
#import matplotlib.pyplot as plt
from matplotlib import pyplot
import pandas as pd
from sklearn import metrics
from collections import Counter
import copy
#from sklearn.model_selection import train_test_split
def data_analysis(dataset):
    numSample = dataset.shape[0]
    dim = dataset.shape[1]
##计算欧式距离
def euclDistance(vector1,vector2):
    return np.sqrt(sum(np.power(vector2-vector1,2)))#power计算次方

##初始化数据的中心点，k表示聚类中心数
##随机生成k个聚类中心
def initCentroids(dataset,k):
    numSample,dim=dataset.shape
    centroids=np.zeros((k,dim))
    for i in range(k):
        #index=int(np.random.uniform(0,numSample))#随机生成数
        index = int(i*12)
        centroids[i,:]=dataset[index,:]
    return centroids

##kmean算法
def kmeans(dataset,k):
    numSample=dataset.shape[0]
    #生成新的两列数组，保存聚类信息
    # 第一列表示所属聚类中心，第二列表示与中心的误差
    clusterAssment=np.zeros((numSample,2))#这里dtype就默认
    clusterChanged=True

## step1 初始化聚类中心
    centroids=initCentroids(dataset,k)
    storage = np.zeros((numSample,))
    itr = 0
    t4centupdate = 0
    t4assment = 0
    while (clusterChanged):
        itr += 1
        #print(itr)
        if(itr == 30):
            
            break
        clusterChanged=False
        
        
        if((storage == clusterAssment[:,0]).all()):
            if((storage == 0).all()):
                clusterChanged = True
            else:
                clusterChanged = False
        else:
            clusterChanged = True
        storage = clusterAssment[:,0]
        #二重循环：对所有数据点，与k个聚类中心计算距离
        #并保存标签与距离
        matrix1 = centroids
        matrix2 = dataset.transpose()
        matrix3 = get_xy_square(dataset, centroids, k)
        distance_matrix = dis_computation(matrix1, matrix2, matrix3)
        
        #np.savetxt("temp.csv",distance_matrix)
        #distance_matrix = np.loadtxt("temp.csv")
        start = time.time()
        clusterAssment = cluster_assignment(distance_matrix)
        end = time.time()
        t4assment += (end-start)

## step4 循环结束后更新聚类中心
        start = time.time()
        for j in range(k):
            #nonzero返回数组中非零元素的位置,
            #eg: clusterAssment[:,0] == j
            #array([ True,  True,  True,  True,  True, False])

            points_In_k_Cluster_Label=np.nonzero(clusterAssment[:,0]==j)[0]

            # pointsInCluster=dataset(np.nonzero(clusterAssment[:,0] == j)[0])
            pointsInCluster=dataset[points_In_k_Cluster_Label]
            centroids[j, :] = np.mean(pointsInCluster, axis=0)
        end = time.time()
        t4centupdate += (end-start)
    print("time for centroids update is " + str(t4centupdate) + " seconds.")
    print("time for assignment is " + str(t4assment)+" seconds.")
    ##循环结束，返回聚类中心和标签信息
    print("Congratulations, cluster complete!")
    return centroids,clusterAssment

if __name__=="__main__":
    # print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    start=time.time()
    ## load data
    dataset=pd.read_csv('yeast.csv',sep=',')
    category_real = dataset.loc[:,["class_protein_localization"]]
    #dataset=pd.read_csv("iris.txt", sep=' ')
    #dataset=dataset.loc[:,["Sepal.Length","Sepal.Width","Petal.Length","Petal.Width"]]
    #dataset=dataset.loc[:, ["region-centroid-col","region-centroid-row","region-pixel-count","short-line-density-5","short-line-density-2","vedge-mean","vegde-sd","hedge-mean","hedge-sd","intensity-mean","rawred-mean","rawblue-mean","rawgreen-mean","exred-mean","exblue-mean","exgreen-mean","value-mean","saturation-mean","hue-mean"]]
    dataset=dataset.loc[:,["mcg","gvh","alm","mit","erl","pox","vac","nuc"]]
    #dataset=dataset.loc[1:25,["Category 1","Category 2","Category 3","Category 4","Category 5","Category 6","Category 7","Category 8","Category 9","Category 10","Category 11","Category 12","Category 13","Category 14","Category 15","Category 16","Category 17","Category 18","Category 19","Category 20","Category 21","Category 22","Category 23","Category 24"]]
    dataset=np.array(dataset)
    #train, test = train_test_split(dataset, test_size = 0.2)
    ##  k表示聚类中心数
    
    k=10
    centroids,clusterAssment=kmeans(dataset,k)

    end = time.time()
    
    print('algorithm (for training) total time: %2f 秒'%(end-start))
    category_real = np.array(category_real)
    category = []
    for i in range(dataset.shape[0]):
        category.append(category_real[i][0])
    category = np.array(category)
    category_pre = np.array(clusterAssment[:,0], dtype = np.int32)
    real = Counter(category)
    pre = Counter(category_pre)
    print(real)
    print(pre)
    real = real.most_common()
    pre = pre.most_common()
    for j in range(dataset.shape[0]):
        for nn in range(k):
            if(category[j] == real[nn][0]):
                category[j] = int(pre[nn][0])
    ARI = metrics.adjusted_rand_score(category, category_pre)
    AMI = metrics.adjusted_mutual_info_score(category, category_pre)
    print("调整兰德指数为" + str(ARI))
    print("归一化互信息指数为" + str(AMI))
    pyplot.scatter(dataset[:, 0], dataset[:, 1], c=clusterAssment[:,0])
    pyplot.scatter(centroids[:, 0], centroids[:, 1], c="white", s=150)
    pyplot.show()